DATASET DOWNLOAD AND EXTRACT

In [1]:
import numpy as np
import tensorflow as tf
from scipy.io import loadmat
from utility import plot_confusion_matrix
import tensorflow.keras.layers as layer
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import time

In [2]:
seed = 1231829
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
from data_loader import get_data
X_data, labels = get_data()

/Users/denisderonjic/Repos/ActivityRecognition/data_loader.py:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return (np.array(X_data), labels)


In [4]:
from data_transformation import frame_data

X_data, y_data = frame_data(X_data, labels)

In [5]:
X_data = X_data[:,:, :6] #Dropping magnetometer

In [6]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

y_data = le.fit_transform(y_data)

In [7]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

num_labels = len(le_name_mapping)
print(num_labels)

{'FALLING': 0, 'JUMPING': 1, 'RUNNING': 2, 'SITTING': 3, 'STNDING': 4, 'WALKING': 5, 'XLYINGX': 6}
7


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

stratifiedSplit = True

if stratifiedSplit:
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=seed)
    for train_index, test_index in split.split(X_data, y_data):
        X_train.append(X_data[train_index])
        y_train.append(y_data[train_index])
        X_test.append(X_data[test_index])
        y_test.append(y_data[test_index])
    X_train = np.array(X_train[0])
    y_train = np.array(y_train[0])
    X_test = np.array(X_test[0])
    y_test = np.array(y_test[0])
else:
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=seed)
    

In [9]:
from data_transformation import data_normalization

X_train = data_normalization(X_train)
X_test = data_normalization(X_test)


2021-09-07 17:39:40.256276: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-07 17:39:40.259004: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [10]:
X_batch = X_test[:100]

Classification NN

In [26]:
model = tf.keras.models.load_model('models/best_CNN_model.h5')

start = time.perf_counter() 
result = model.predict(X_batch)
end = time.perf_counter()

print(end - start)

0.07574608400000216


Inception Network

In [32]:
model = tf.keras.models.load_model('models/best_Inception_Raw_DCT_model.h5')

start = time.perf_counter() 
result = model.predict(X_batch)
end = time.perf_counter()

print(end - start)

ValueError: Unknown layer: InceptionBlockV2. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [31]:
model = tf.keras.models.Sequential([tf.keras.models.load_model('models/best_SSAE_model_encoder.h5'), tf.keras.models.load_model('models/best_SSAE_model_classifier.h5')])

start = time.perf_counter() 
result = model.predict(X_batch)
end = time.perf_counter()

print(end - start)

0.0614620000000059
